In [1]:
import pandas as pd
import torch
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np
%matplotlib inline

In [2]:
import pandas as pd

class TimeSeriesLabeler:
    def __init__(self, term):
        self.term = term
        self.result_df = pd.DataFrame()
        self.labels = []

    def calculate_labels(self, df):
        for id in df['ID'].unique():
            df_id = df[df['ID'] == id].reset_index(drop=True)
            label = []

            for i in range(self.term, len(df_id)):
                previous_days = df_id['weight'].iloc[i - self.term:i]
                current_weight = df_id['weight'].iloc[i]
                if current_weight > round(previous_days.mean(),3):
                    label.append(1)
                elif current_weight == round(previous_days.mean(),3):
                    label.append(0)
                elif current_weight < round(previous_days.mean(),3):
                    label.append(2)
                else:
                    print("error")

            self.labels.extend(label)

    def create_result_df(self, df):
        for id in df['ID'].unique():
            df_id = df[df['ID'] == id].reset_index(drop=True)
            df_id_drop = df_id.drop(index=range(self.term))
            self.result_df = pd.concat([self.result_df, df_id_drop])
        self.result_df['labels']= self.labels
        
    def get_labels(self):
        return self.labels

    def get_result_df(self):
        return self.result_df

# 데이터 불러오기

### 남자 여자

In [3]:
df_boy=pd.read_csv("./의미 있는 data/df_boy.csv",encoding="cp949")
df_girl=pd.read_csv("./의미 있는 data/df_girl.csv",encoding="cp949")

### 학년별

In [4]:
df_grade4=pd.read_csv("./의미 있는 data/df_grade4.csv",encoding="cp949")
df_grade5=pd.read_csv("./의미 있는 data/df_grade5.csv",encoding="cp949")
df_grade6=pd.read_csv("./의미 있는 data/df_grade6.csv",encoding="cp949")

### 보상별

In [5]:
reward_1=pd.read_csv("./의미 있는 data/reward_1.csv",encoding="cp949")
reward_2=pd.read_csv("./의미 있는 data/reward_2.csv",encoding="cp949")
reward_3=pd.read_csv("./의미 있는 data/reward_3.csv",encoding="cp949")

### 저체중, 정상체중, 과체중

In [6]:
df_thin=pd.read_csv("./의미 있는 data/df_thin.csv",encoding="cp949")
df_origin=pd.read_csv("./의미 있는 data/df_origin.csv",encoding="cp949")
df_over=pd.read_csv("./의미 있는 data/df_over.csv",encoding="cp949")

## ID 별로 일주일 간격으로 몸무게 변화 라벨링(유지:0, 감소:2, 증가:1)

In [7]:
df_list=[df_boy,df_girl,df_grade4,df_grade5,df_grade6,reward_1,reward_2,reward_3,df_thin,df_origin,df_over]

In [8]:
term = 7  # You can set your desired 'term' here

In [9]:
df_boy

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부,BMI
0,#AJXZPA,2022-07-14,1,5,136.1,31.8,3579.0,53.55,701.0,480.0,2,17.167651
1,#AJXZPA,2022-07-15,1,5,136.1,31.8,3579.0,53.55,701.0,480.0,2,17.167651
2,#AJXZPA,2022-07-16,1,5,136.1,31.8,3579.0,53.55,701.0,480.0,2,17.167651
3,#AJXZPA,2022-07-17,1,5,136.1,31.8,3579.0,53.55,701.0,480.0,2,17.167651
4,#AJXZPA,2022-07-18,1,5,136.1,31.8,3579.0,53.55,701.0,480.0,2,17.167651
...,...,...,...,...,...,...,...,...,...,...,...,...
23655,#ZZLSSL,2023-01-07,1,6,153.0,68.0,3579.0,53.55,701.0,480.0,1,29.048656
23656,#ZZLSSL,2023-01-08,1,6,153.0,68.0,3579.0,53.55,701.0,480.0,1,29.048656
23657,#ZZLSSL,2023-01-09,1,6,153.0,68.0,3579.0,53.55,701.0,480.0,1,29.048656
23658,#ZZLSSL,2023-01-10,1,6,153.0,68.0,3579.0,53.55,701.0,480.0,1,29.048656


In [10]:
df_thin

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부,BMI
0,#BBLXZS,2022-07-14,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
1,#BBLXZS,2022-07-15,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
2,#BBLXZS,2022-07-16,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
3,#BBLXZS,2022-07-17,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
4,#BBLXZS,2022-07-18,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
...,...,...,...,...,...,...,...,...,...,...,...,...
5091,#ZFPNFC,2023-01-07,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
5092,#ZFPNFC,2023-01-08,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
5093,#ZFPNFC,2023-01-09,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
5094,#ZFPNFC,2023-01-10,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049


In [11]:
df_thin

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부,BMI
0,#BBLXZS,2022-07-14,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
1,#BBLXZS,2022-07-15,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
2,#BBLXZS,2022-07-16,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
3,#BBLXZS,2022-07-17,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
4,#BBLXZS,2022-07-18,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
...,...,...,...,...,...,...,...,...,...,...,...,...
5091,#ZFPNFC,2023-01-07,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
5092,#ZFPNFC,2023-01-08,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
5093,#ZFPNFC,2023-01-09,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
5094,#ZFPNFC,2023-01-10,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049


In [12]:
for i,df in enumerate(df_list):
    variable_name=f"labeler_{i}"
    globals()['labeler_{}'.format(i)]= TimeSeriesLabeler(term)
    print(i)
    eval(variable_name).calculate_labels(df)  # Provide your DataFrame 'df' as input
    eval(variable_name).create_result_df(df)  # Create the result DataFrame
    labels = eval(variable_name).get_labels()
    globals()['result_{}'.format(i)]= eval(variable_name).get_result_df()

0
1
2
3
4
5
6
7
8
9
10


# 데이터 세트 로드

In [13]:
result_df_list=[result_0,result_1,result_2,result_3,result_4,result_5,result_6,result_7,result_8,result_9,result_10]

In [14]:
for i,result_df in enumerate(result_df_list):
    globals()['X_{}'.format(i)]=result_df.iloc[:,[2,3,4,5,6,7,8,9,10]]
    globals()['y_{}'.format(i)]=result_df.iloc[:,-1].to_frame()

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
for i in range(len(result_df_list)):
    globals()['scaler_{}'.format(i)]=StandardScaler()
    variable_name=f"scaler_{i}"
    X_name=f"X_{i}"
    globals()['X_scaler_{}'.format(i)]=eval(variable_name).fit_transform(eval(X_name))

# 데이터 스케일링

## Train, Test dataset 나누기

In [16]:
for i in range(len(result_df_list)):
    globals()['X_train_{}'.format(i)], globals()['X_test_{}'.format(i)] , globals()['y_train_{}'.format(i)] , globals()['y_test_{}'.format(i)]  = train_test_split(X_scaler_1, y_1, test_size=0.20)

## Global train 데이터 세트 만들기

In [17]:
X_train_list=[]
y_train_list=[]

for i in range(len(result_df_list)):
    X_train_list.append(eval(f"X_train_{i}"))
    y_train_list.append(eval(f"y_train_{i}"))
X_train = np.concatenate(X_train_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)

In [18]:
X_test_list=[]
y_test_list=[]
for i in range(len(result_df_list)):
    X_test_list.append(eval(f"X_test_{i}"))
    y_test_list.append(eval(f"y_test_{i}"))
X_test = np.concatenate(X_test_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)

In [19]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
train_dataset=TensorDataset(X_train, y_train)
test_dataset=TensorDataset(X_test, y_test)

## Client train 데이터 세트 만들기

In [20]:
for i in range(len(result_df_list)):
    globals()['X_train_{}'.format(i)]=torch.FloatTensor(eval(f"X_train_{i}"))
    globals()['X_test_{}'.format(i)]=torch.FloatTensor(eval(f"X_test_{i}"))
    globals()['y_train_{}'.format(i)]=torch.LongTensor(eval(f"y_train_{i}").to_numpy())
    globals()['y_test_{}'.format(i)]=torch.LongTensor(eval(f"y_test_{i}").to_numpy())
    globals()['train_dataset_{}'.format(i)]=TensorDataset(eval(f"X_train_{i}"), eval(f"y_train_{i}"))
    globals()['test_dataset_{}'.format(i)]=TensorDataset(eval(f"X_test_{i}"), eval(f"y_test_{i}"))

# 파라미터 값 정의

In [21]:
total_train_size=len(train_dataset)


total_train_size_1 = len(train_dataset_1)
total_test_size_1 = len(test_dataset_1)

total_train_size_2 = len(train_dataset_2)
total_test_size_2 = len(test_dataset_2)

total_train_size_3 = len(train_dataset_3)
total_test_size_3 = len(test_dataset_3)

classes = 3
input_dim = 9

num_clients = len(result_df_list)
rounds = 10
batch_size = 7000
epochs_per_client = 10
learning_rate = 2e-2

# GPU 설정

In [22]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader(DataLoader):
        def __init__(self, dl, device):
            self.dl = dl
            self.device = device

        def __iter__(self):
            for batch in self.dl:
                yield to_device(batch, self.device)

        def __len__(self):
            return len(self.dl)

device = get_device()

# 딥러닝 모델 클래스 정의

In [23]:
class DNNModel(torch.nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        self.input_layer = nn.Linear(input_dim, 128)
        self.hidden_layer1 = nn.Linear(128, 256)
        self.hidden_layer2 = nn.Linear(256, 128)
        self.output_layer   = nn.Linear(128,3)
        self.relu = nn.ReLU()
        self.track_layers = {'hidden_layer1': self.hidden_layer1, 'hidden_layer2': self.hidden_layer2, 'output_layer': self.output_layer}

    def forward(self, x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.relu(self.hidden_layer2(out))
        out =  self.output_layer(out)
        return out 
    
    def get_track_layers(self):
        return self.track_layers
    
    def apply_parameters(self, parameters_dict):
        with torch.no_grad():
            for layer_name in parameters_dict:
                self.track_layers[layer_name].weight.data *= 0
                self.track_layers[layer_name].bias.data *= 0
                self.track_layers[layer_name].weight.data += parameters_dict[layer_name]['weight']
                self.track_layers[layer_name].bias.data += parameters_dict[layer_name]['bias']
    
    def get_parameters(self):
        parameters_dict = dict()
        for layer_name in self.track_layers:
            parameters_dict[layer_name] = {
                'weight': self.track_layers[layer_name].weight.data, 
                'bias': self.track_layers[layer_name].bias.data
            }
        return parameters_dict
    
    def batch_accuracy(self, outputs, labels):
        with torch.no_grad():
            _, predictions = torch.max(outputs, dim=1)
            return torch.tensor(torch.sum(predictions == labels).item() / len(predictions))
    
    def _process_batch(self, batch):
        
        data, labels = batch
        outputs= self(data)
        loss = torch.nn.functional.cross_entropy(outputs, labels.squeeze(dim=-1))
        accuracy = self.batch_accuracy(outputs, labels.squeeze(dim=-1))
        return (loss, accuracy)
    
    def fit(self, dataset, epochs, lr, batch_size=128, opt=torch.optim.SGD):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size, shuffle=True), device)
        optimizer = opt(self.parameters(), lr)
        history = []
        for epoch in range(epochs):
            losses = []
            accs = []
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss.detach()
                losses.append(loss)
                accs.append(acc)
            avg_loss = torch.stack(losses).mean().item()
            avg_acc = torch.stack(accs).mean().item()
            history.append((avg_loss, avg_acc))
        return history
    
    def evaluate(self, dataset, batch_size=128):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size), device)
        losses = []
        accs = []
        with torch.no_grad():
            for batch in dataloader:
                loss, acc = self._process_batch(batch)
                losses.append(loss)
                accs.append(acc)
        avg_loss = torch.stack(losses).mean().item()
        print("evaluate_losses.,shape",np.array(losses).shape)
        print("evaluate_avg_loss",avg_loss)
        avg_acc = torch.stack(accs).mean().item()
        return (avg_loss, avg_acc)

# 클라이언트 클래스

In [24]:
class Client:
    def __init__(self, client_id, dataset):
        self.client_id = client_id
        self.dataset = dataset
    
    def get_dataset_size(self):
        return len(self.dataset)
    
    def get_client_id(self):
        return self.client_id
    
    def train(self, parameters_dict):
        net = to_device(DNNModel(), device)
        net.apply_parameters(parameters_dict)
        train_history = net.fit(self.dataset, epochs_per_client, learning_rate, batch_size)
        print('{}: Loss = {}, Accuracy = {}'.format(self.client_id, round(train_history[-1][0], 4), round(train_history[-1][1], 4)))
        return net.get_parameters()

# 클라이언트 셋업

In [25]:
train_dataset_list=[]
for i in range(len(result_df_list)):
    train_dataset_list.append(eval(f"train_dataset_{i}"))
client_datasets=train_dataset_list

clients = [Client('client_' + str(i), client_datasets[i]) for i in range(num_clients)]

# 중앙 서버 시작

In [26]:
global_net = to_device(DNNModel(), device)
history = []
j=0
for i in range(rounds):
    print('Start Round {} ...'.format(i + 1))
    curr_parameters = global_net.get_parameters()
    new_parameters = dict([(layer_name, {'weight': 0, 'bias': 0}) for layer_name in curr_parameters])
    for index, client in enumerate(clients):
    
        client_parameters = client.train(curr_parameters)
        fraction = client.get_dataset_size() / total_train_size
        for layer_name in client_parameters:
            new_parameters[layer_name]['weight'] += fraction * client_parameters[layer_name]['weight']
            new_parameters[layer_name]['bias'] += fraction * client_parameters[layer_name]['bias']
    global_net.apply_parameters(new_parameters)
    
    train_loss, train_acc = global_net.evaluate(train_dataset)
    dev_loss, dev_acc = global_net.evaluate(test_dataset)
    print('After round {}, train_loss = {}, dev_loss = {}, ,train_acc={}, dev_acc = {}\n'.format(i + 1, round(train_loss, 4), 
            round(dev_loss, 4),round(train_acc, 4) ,round(dev_acc, 4)))
    history.append((train_loss, dev_loss))

Start Round 1 ...
client_0: Loss = 0.7015, Accuracy = 0.884
client_1: Loss = 0.6308, Accuracy = 0.8841
client_2: Loss = 0.6943, Accuracy = 0.8838
client_3: Loss = 0.6883, Accuracy = 0.8833
client_4: Loss = 0.6978, Accuracy = 0.8834
client_5: Loss = 0.6599, Accuracy = 0.883
client_6: Loss = 0.6618, Accuracy = 0.8844
client_7: Loss = 0.6726, Accuracy = 0.8835
client_8: Loss = 0.6708, Accuracy = 0.8858
client_9: Loss = 0.676, Accuracy = 0.8846
client_10: Loss = 0.649, Accuracy = 0.8844
evaluate_losses.,shape (2238,)
evaluate_avg_loss 0.6826261281967163
evaluate_losses.,shape (560,)
evaluate_avg_loss 0.6815245747566223
After round 1, train_loss = 0.6826, dev_loss = 0.6815, ,train_acc=0.8839, dev_acc = 0.884

Start Round 2 ...
client_0: Loss = 0.5099, Accuracy = 0.8841
client_1: Loss = 0.5218, Accuracy = 0.884
client_2: Loss = 0.5132, Accuracy = 0.8839
client_3: Loss = 0.5222, Accuracy = 0.8833
client_4: Loss = 0.5219, Accuracy = 0.8828
client_5: Loss = 0.5102, Accuracy = 0.883
client_6: Lo